In [1]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split


c:\Users\Edward\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('protein_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Gene A Sequence,Gene B Sequence,Interaction Type,Detection Method
0,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Two hybrid
1,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Pull down
2,MQRLKKFIAKREKGDKGKMKWNSSMDYDSPPSYQDVRRGIFPTAPL...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Physical association,Enzyme linked immunosorbent assay
3,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Direct interaction,Molecular sieving
4,MTAKMETTFYDDALNASFLPSESGPYGYSNPKILKQSMTLNLADPV...,MSSTLAKIAEIEAEMARTQKNKATAHHLGLLKARLAKLRRELITPK...,Association,Anti bait coimmunoprecipitation


In [3]:
df.shape

(100947, 4)

In [4]:
# removing nan
df = df.dropna()

In [5]:
df.shape

(99860, 4)

In [6]:
counts = {}
mapping = {}
allData = []

check = set()
for index, row in df.iterrows():
    aSeq = row['Gene A Sequence']
    bSeq = row['Gene B Sequence']

    counts[aSeq] = counts.get(aSeq, 0) + 1
    counts[bSeq] = counts.get(bSeq, 0) + 1

    if aSeq in mapping:
        mapping[aSeq].add(bSeq)
    else:
        mapping[aSeq] = {bSeq}

    if bSeq in mapping:
        mapping[bSeq].add(aSeq)
    else:
        mapping[bSeq] = {aSeq}
    
    data = (aSeq, bSeq, 1)
    
    if data in check: continue
    
    check.add(data)
    allData.append(data)

In [7]:
allGenes = set(df["Gene A Sequence"]).union(set(df["Gene B Sequence"]))

In [8]:
def getAllGenesInteracted(gene):
  return mapping[gene]


In [9]:
def getAllGenesNotInteracted(gene):
  genesInteracted = getAllGenesInteracted(gene)
  return allGenes - genesInteracted

In [10]:
negativeSamples = []

In [ ]:
# takes around 4 minutes to negative sample
from collections import defaultdict

gene_ctr = defaultdict(int)

for geneA, geneB, _ in allData:

  genesNotInteractedA = getAllGenesNotInteracted(geneA)
  genesNotInteractedB = getAllGenesNotInteracted(geneB)
  
  geneA_idx = gene_ctr[geneA]
  geneB_idx = gene_ctr[geneB]
  
  randomGeneA = list(genesNotInteractedA)[geneA_idx]
  randomGeneB = list(genesNotInteractedB)[geneB_idx]

  negativeSampleA = (geneA, randomGeneA, 0)
  negativeSampleB = (geneB, randomGeneB, 0)
  
  gene_ctr[geneA] = (gene_ctr[geneA]+1)%len(genesNotInteractedA)
  gene_ctr[geneB] = (gene_ctr[geneB]+1)%len(genesNotInteractedB)
  
  negativeSamples.append(negativeSampleA)
  negativeSamples.append(negativeSampleB)


In [12]:
len(allData)

89163

In [13]:
allData = allData + negativeSamples

In [14]:
df = pd.DataFrame(allData, columns=["GeneA", "GeneB", "Interaction"])



In [15]:
df.shape

(267489, 3)

In [16]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
dfTrain = pd.DataFrame(train, columns=["GeneA", "GeneB", "Interaction"])
dfTest = pd.DataFrame(test, columns=["GeneA", "GeneB", "Interaction"])

dfTrain.to_csv('train.csv', index=False)
dfTest.to_csv('test.csv', index=False)